# 0. Instalar e importar bibliotecas 📚

In [ ]:
!pip install requests
!pip install requests_oauthlib
!pip install oauthlib
!pip install requests
!pip install pandas

In [1]:
import requests
from requests_oauthlib import OAuth2Session
from requests_oauthlib import OAuth2Session
from requests.auth import HTTPBasicAuth
from oauthlib.oauth2 import BackendApplicationClient
import json

In [2]:
from datetime import datetime as dt
import pandas as pd


# 1. Obter token de acesso 🔑

Usar o arquivo json gerado no notebook credentials com client_id e client_secret.

In [3]:
keys = json.loads(open('keys.json', 'r').read())

In [4]:
client_id = keys['client_id']
client_secret = keys['client_secret']
scope= "accountability.statements accountability.request"

In [5]:
auth = HTTPBasicAuth(client_id, client_secret)
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client) 
oauth = OAuth2Session(client=client, scope=scope)
token = oauth.fetch_token(token_url='https://oauth.hm.bb.com.br/oauth/token', auth=auth)

In [7]:
print('validade do token: ', dt.strftime(dt.fromtimestamp(token['expires_at']), '%d/%m/%y %H:%M:%S'))

validade do token:  12/07/23 17:01:51


# 2. Requisições 📞

Criar cabeçalho com a chave da aplicação de homologação e token de acesso.

In [8]:
gw_dev_app_key = "substituir pela chave da aplicação"

headers = {
            'accept': 'application/json',
            'Authorization': 'Bearer '+token['access_token'],
            'X-Application-Key' : gw_dev_app_key
            }

## 2.1. Agências próximas 🏦

In [10]:
cnpj = '11435633000149'
cep = '50050910'

r = requests.get(f"https://api.hm.bb.com.br/accountability/v3/agencias-proximas?cnpj={cnpj}&cep={cep}", headers=headers)
r.json()

{'quantidadeAgencia': 6,
 'listaAgencia': [{'codigo': 697,
   'digito': '1',
   'nome': 'SETE DE SETEMBRO',
   'cep': '50060070',
   'logradouro': 'R.SETE DE SETEMBRO,128',
   'bairro': 'BOA VISTA',
   'municipio': 'RECIFE',
   'siglaUF': 'PE',
   'sugerida': 'N'},
  {'codigo': 7,
   'digito': '8',
   'nome': 'RECIFE',
   'cep': '50030310',
   'logradouro': 'AV.RIO BRANCO,240',
   'bairro': 'RECIFE ANTIGO',
   'municipio': 'RECIFE',
   'siglaUF': 'PE',
   'sugerida': 'N'},
  {'codigo': 1814,
   'digito': '7',
   'nome': 'SHOPPING TACARUNA',
   'cep': '50110000',
   'logradouro': 'AV.AGAMENON MAGALHAES,153',
   'bairro': 'STO.AMARO',
   'municipio': 'RECIFE',
   'siglaUF': 'PE',
   'sugerida': 'N'},
  {'codigo': 2337,
   'digito': 'X',
   'nome': 'NEOPOLIS',
   'cep': '49980000',
   'logradouro': 'PCA.MONS.JOSE M.DE SANTANA,27',
   'bairro': 'CENTRO',
   'municipio': 'NEOPOLIS',
   'siglaUF': 'SE',
   'sugerida': 'N'},
  {'codigo': 2206,
   'digito': '3',
   'nome': 'JAPARATUBA',
   'ce

## 2.2. Extratos

In [14]:
agencia = "1"
conta = "1"
data_inicio = "2023-01-01"
data_fim = "2023-02-01"

r = requests.get(f"https://api.hm.bb.com.br/accountability/v3/statements/{agencia}-{conta}/control-agencies?startDate={data_inicio}&endDate={data_fim}", headers=headers)
r.json()

{'error': 'Agência e Conta não está vinculado ao Órgão de Controle.'}